In [1]:
from urllib2 import Request, urlopen, URLError
from bs4 import BeautifulSoup
import re
import turtle
import time
import numpy as np
import math
#import matplotlib.pyplot as plt
from scipy.interpolate import Rbf
#from matplotlib import cm
PI = 3.1415926
classification = [[1,2,3,4],5,[6,7],[8,11],9,10,12,[13,14,16],15,17,18,20]#POI classification
para = ['AQI','NO2','PM25','PM10','CO','O3','SO2','Wind','Pressure','Temperature','Humidity']

In [ ]:
def POIBasedInterpolation(Coordinate,Data,currentC):
    Dist = list()
    #DIS = 0
    d = 0
    #Dist_copy = list()
    POIC = getPOIsFromGPS(currentC[0],currentC[1],1000)
    for cor in Coordinate:
        POI = getPOIsFromGPS(cor[0],cor[1],1000)
        dis = (np.sqrt(((cor - currentC)**2)))**(-1)
        Dist.append(dis)
        #DIS = DIS + dis
    for i in range(0,len(Coordinate)):
        #print Dist[i]
        d = d + (Dist[i]/DIS)*Data[i]
    return d

In [2]:
def getOnlineDataFromMS(Coordinate):
    Data = list()
    for i in range(0,11):
        l = list()
        Data.append(l)
    for cor in Coordinate:
        URL = 'http://urbanair.msra.cn/U_Air/SearchGeoPoint?Latitude='+str(cor[0])+'&Longitude='+str(cor[1])+'&Standard=0&Culture=zh-CN'
        request = Request(URL)
        try:
            response = urlopen(request)
            kittens = response.read()
            llist = re.split(':',kittens);
            #print kittens
            for i in range(1,8):
                a = re.split(',',llist[i])
                if a[0] == 'null':
                    Data[i-1].append(None)
                else:
                    Data[i-1].append(float(a[0]))
                #print type(kittens)
            for i in range(10,14):
                a = re.split(',',llist[i])
                if a[0] == 'null':
                    Data[i-3].append(None)
                else:
                    Data[i-3].append(float(a[0]))
                #print type(kittens)
        except URLError, e:
            print 'Got an error code:', e
    return Data

In [3]:
def getPredictPM25FromGPS(x,y):
    URL = 'http://urbanair.msra.cn/U_Air/SearchPrediction_48V2?Latitude=' + str(x) + '&Longitude=' + str(y) + '&Pollutant=PM25' 
    request = Request(URL)
    predict = []
    try:
        response = urlopen(request)
        kittens = response.read()
        llist = re.split(':',kittens);
        llist = re.split(',',llist[1])
        predict.append(int(llist[0][1:]))
        for i in range(1,13):
            predict.append(int(llist[i]))
        predict.append(int(llist[13][:-1]))
    except URLError, e:
        print 'Got an error code:', e
    return predict#[0:7定时预测，8:13三个区间]

In [4]:
def getTrafficFromGPS(x,y,r):
    URL = 'http://restapi.amap.com/v3/traffic/status/circle?key=b2a732f0d7cddfcdc81bee76938e34c2&location='+str(y)+','+str(x)+'&radius='+str(r)+'&output=xml&offset=20&page=1&extensions=all' 
    request = Request(URL)
    try:
        response = urlopen(request)
        kittens = response.read()
    except URLError, e:
        print 'Got an error code:', e
    soup = BeautifulSoup(kittens,"html.parser")
    trafficV = np.zeros(4)
    #print type(soup.evaluation.contents)
    if soup.evaluation==None:
        print("No Data")
        return False
    if soup.evaluation.expedite.contents==[]:
        print ("No Data")
        return False
    #print soup.evaluation.congested.contents
    #print soup.evaluation.blocked.contents
    #print soup.evaluation.unknown.contents
    trafficV[0] = float(soup.evaluation.expedite.contents[0][:4])
    trafficV[1] = float(soup.evaluation.congested.contents[0][:4])
    trafficV[2] = float(soup.evaluation.blocked.contents[0][:4])
    trafficV[3] = float(soup.evaluation.unknown.contents[0][:4])
    return trafficV

In [5]:
def getPOIsFromGPS(x,y,r):
    URL = 'http://restapi.amap.com/v3/place/around?key=b2a732f0d7cddfcdc81bee76938e34c2&location='+str(y)+','+str(x)+'&output=xml&radius='+str(r)+'&offset=20&page=1&extensions=all'
    request = Request(URL)
    try:
        response = urlopen(request)
        kittens = response.read()
    except URLError, e:
        print 'Got an error code:', e
    soup = BeautifulSoup(kittens,"html.parser")
    a = soup.pois
    poiV = np.zeros(12)
    for child in a:
        b = int(child.typecode.contents[0])
        if b/10000>0 and b/10000<5:
            poiV[0] = poiV[0]+1
        elif b/10000==5:
            poiV[1] = poiV[1]+1
        elif b/10000==6 or b/10000==7:
            poiV[2] = poiV[2]+1
        elif b/10000==8 or b/10000==11:
            poiV[3] = poiV[3]+1
        elif b/10000==9:
            poiV[4] = poiV[4]+1
        elif b/10000==10:
            poiV[5] = poiV[5]+1
        elif b/10000==12:
            poiV[6] = poiV[6]+1
        elif b/10000==13 or b/10000==14 or b/10000==16:
            poiV[7] = poiV[7]+1
        elif b/10000==15:
            poiV[5] = poiV[8]+1
        elif b/10000==17:
            poiV[5] = poiV[9]+1
        elif b/10000==18:
            poiV[5] = poiV[10]+1
        elif b/10000==20:
            poiV[5] = poiV[11]+1
    return poiV

In [6]:
def getLocationFromKeywords(keywords):
    URL = 'http://restapi.amap.com/v3/place/text?&keywords=' + keywords + '&city=beijing&output=xml&offset=20&page=1&key='+'b2a732f0d7cddfcdc81bee76938e34c2' + '&extensions=all'
    request = Request(URL)
    try:
        response = urlopen(request)
        kittens = response.read()
    except URLError, e:
        print 'Got an error code:', e
    soup = BeautifulSoup(kittens,"html.parser")
    Str = soup.location.string
    lstr = re.split(',',Str)
    lon = float(lstr[0])
    lat = float(lstr[1])
    return lat,lon

In [7]:
def DistantBasedInterpolation(Coordinate,Data,currentC):
    Dist = list()
    DIS = 0
    d = 0
    #Dist_copy = list()
    for cor in Coordinate:
        dis = (np.sqrt(((cor[0] - currentC[0])**2+(cor[1]-currentC[1])**2)))**(-1)
        Dist.append(dis)
        DIS = DIS + dis
    for i in range(0,len(Coordinate)):
        #print Dist[i]
        d = d + (Dist[i]/DIS)*Data[i]
    return d

In [8]:
def compare(a,b):
    x=[a[0]-p0[0],a[1]-p0[1]]
    y=[b[0]-p0[0],b[1]-p0[1]]
    dx=(x[0]**2+x[1]**2)**0.5
    dy=(y[0]**2+y[1]**2)**0.5
    cosa=x[0]/dx
    cosb=y[0]/dy
    if cosa < cosb:
        return 1
    elif cosa > cosb:
        return -1
    else:
        if dx<dy:
            return -1
        elif dx>dy:
            return 1
        else:
            return 0

In [9]:
def windingNumber(p, poly):
    sum = 0
    px = p[0]
    py = p[1]
    print px,py
    l = len(poly)
    for i in range(0,l):
        if i == 0:
            j = l-1
        else:
            j = i-1
        sx = poly[i][0]
        sy = poly[i][1]
        tx = poly[j][0]
        ty = poly[j][1]
        #点与多边形顶点重合或在多边形的边上
        if (sx - px) * (px - tx) >= 0 and (sy - py) * (py - ty) >= 0 and (px - sx) * (ty - sy) == (py - sy) * (tx - sx):
            return True
        #点与相邻顶点连线的夹角
        angle = math.atan2(sy - py, sx - px) - math.atan2(ty - py, tx - px)
        #确保夹角不超出取值范围（-π 到 π）
        if angle >= PI:
            angle = angle - PI * 2
        elif angle <= -PI: 
            angle = angle + PI * 2
        sum =sum + angle
        #计算回转数并判断点和多边形的几何关系
        if sum / PI >= -0.01 and sum / PI <= 0.01:
            return False
        else:
            return True

In [10]:
def ScatterInterpolation(Coordinate,currentC,Data):
    x = []
    y = []
    for i in range(0,len(Coordinate)):
        x.append(Coordinate[i][0])
        y.append(Coordinate[i][1])
    x = np.asarray(x)
    y = np.asarray(y)
    z = np.asarray(Data)
    rbf = Rbf(x, y, z, epsilon=1)
    ZI = rbf(currentC[0], currentC[1])
    return ZI

In [11]:
def TriangleInterpolation(x1,y1,x2,y2,x3,y3,x,y,a,b,c):
    d = a + (b-a)*((x-x1)*(y3-y1)+(y-y1)*(x1-x3))/((y3-y1)*(x2-x1)+(y1-y2)*(x3-x1)) +\
    (c-a)*((y-y1)*(x2-x1)+(y1-y2)*(x-x1))/((y3-y1)*(x2-x1)+(y1-y2)*(x3-x1))
    return d    

In [12]:
def FindThreePoints(Coordinate,currentC):
    Dist = list()
    #Dist_copy = list()
    for cor in Coordinate:
        dis = np.sqrt(((cor[0] - currentC[0])**2+(cor[1]-currentC[1])**2))
        Dist.append(dis)
        #Dist_copy.append(dis)
    MAX = max(Dist)
    p1 = Dist.index(min(Dist))
    Dist[p1] = MAX
    p2 = Dist.index(min(Dist))
    Dist[p2] = MAX
    p3 = Dist.index(min(Dist))
    #return Dist_copy[p1],Dist_copy[p2],Dist_copy[p3]
    return Coordinate[p1],Coordinate[p2],Coordinate[p3]

In [13]:
#get coordinate of 36 positions
def getCoordinate():
    file_ads = "/Users/Claire/Desktop/intern/20170317/UrbanAir_get/airStation.txt" 
    f = open(file_ads,"r");
    line = f.readline();
    point = list()
    Coordinate = list()
    while line:
    #print line
        llist = re.split('=',line);
        x = re.split('&',llist[1])
        y = re.split('&',llist[2])
        cor = [float(x[0]),float(y[0])]
        point.append(cor)
        Coordinate.append(cor)
        line = f.readline();
    return point, Coordinate

In [ ]:
def getCircle(point):
    n=0
    for i in range(len(point)):
        if point[n][1]>point[i][1]:
            n=i
    p0=point[n]
    point.pop(n)
    point.sort(compare)
    point.insert(0,p0)
    ep=point[:]#复制元素，操作ep不会对point产生影响
    tag=0
    while tag==0:
        tag=1
        l=len(ep)
        for i in range(l):
            i1,i2,i3=(i,(i+1)%l,(i+2)%l)
            x,y,z=(ep[i1],ep[i2],ep[i3])
            a1,a2=((y[0]-x[0],y[1]-x[1]),(z[0]-y[0],z[1]-y[1]))
            if a1[0]*a2[1]-a1[1]*a2[0] < 0:
                tag=0
                ep.pop(i2)
                break
            elif a1[0]*a2[1]-a1[1]*a2[0]==0 and a1[0]*a2[0]<0:
            #==0应改写,360度的情况
                tag=0
                ep.pop(i2)
                break
    return ep;

In [ ]:
def getDataForKeywords(keywords):
    point,Coordinate = getCoordinate()
    x,y = getLocationFromKeywords("北京大学")
    p = [x,y]
    Data = getOnlineDataFromMS(Coordinate)
    if windingNumber(p,Coordinate):
        d = DistantBasedInterpolation(Coordinate,Data[0],p)
    else:
        poiV = getPOIsFromGPS(x,y,1000)
        TrafficV = getTrafficFromGPS(x,y,1000)

In [14]:
x,y = getLocationFromKeywords("北京大学")
p = [x,y]
Data = getOnlineDataFromMS(Coordinate)
if windingNumber(p,Coordinate):
    d = DistantBasedInterpolation(Coordinate,Data[0],p)
else:
    poiV = getPOIsFromGPS(x,y,1000)
    TV = getTrafficFromGPS(x,y,1000)

39.99281 116.31088


In [16]:
for cor in Coordinate:
    getTrafficFromGPS(cor[0],cor[1],1000)

No Data
No Data
No Data
No Data
No Data
No Data
No Data
No Data
